In [1]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns

from tqdm.notebook import tqdm

In [2]:
# result_dir = "/Users/chanb/research/ualberta/Avg_OPE/cc_results/results/disc_cop"
result_dir = "/Users/chanb/research/ualberta/Avg_OPE/cc_results/hyperparameter_tune/results"
env_name = "cartpole"

In [6]:
run_data

{'seeds': range(0, 10),
 'env_name': 'cartpole',
 'hyperparameters': {'lr': 0.0001,
  'env': 'CartPole-v1',
  'policy_path': '/home/chanb/src/Avg_OPE/exper/cartpole.pth',
  'link': 'default',
  'random_weight': 0.5,
  'l1_lambda': 0.1,
  'checkpoint': 5,
  'epoch': 2000,
  'cv_fold': 1,
  'batch_size': 256,
  'max_ep': 40,
  'max_len': 100,
  'use_batch_norm': True,
  'use_target_network': False,
  'discount': 0.95,
  'cop_discount': 0.95,
  'tau': 0.0005,
  'baseline_path': '/home/chanb/scratch/disc_cop/results/baseline-cartpole.pkl',
  'save_path': '/home/chanb/scratch/disc_cop/results/saved_models/cartpole'},
 'results': {}}

In [5]:
baseline = pickle.load(open(os.path.join(result_dir, "baseline-{}.pkl".format(env_name)), "rb"))

all_runs = []
all_errors = []
for run_file in tqdm(os.listdir(os.path.join(result_dir, env_name))):
    run_data = pickle.load(open(os.path.join(result_dir, env_name, run_file), "rb"))

    # if len(run_data["results"]) == 0:
    #     print(run_file)
    #     continue

    all_runs.append(run_file.split(".pkl")[0])
    all_errors.append([])

    for seed in run_data["seeds"]:
        all_errors[-1].append((np.array(run_data["results"][seed][1]) - baseline[seed][run_data["hyperparameters"]["discount"]][0]) ** 2)

  0%|          | 0/48 [00:00<?, ?it/s]

KeyError: 0

In [13]:
all_runs = np.array(all_runs)

In [14]:
all_errors = np.array(all_errors)

In [15]:
mse_per_run = np.mean(all_errors, axis=(1, 2))

In [16]:
sort_idxes = np.argsort(mse_per_run)

In [ ]:
mse_per_run[sort_idxes]

In [ ]:
top_k = 5

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
for run_name, errors in zip(all_runs[sort_idxes][:top_k], all_errors[sort_idxes][:top_k]):
    print(run_name)

    # log_errors = np.log10(errors)
    log_errors = (errors)
    mean_errors = np.mean(log_errors, axis=0)
    std_errors = np.std(log_errors, axis=0) / np.sqrt(len(log_errors))
    ax.plot(np.arange(errors.shape[1]), mean_errors)
    ax.fill_between(np.arange(errors.shape[1]), mean_errors + std_errors, mean_errors - std_errors, alpha=0.2)
fig.tight_layout()
plt.show()